In [1]:
import pandas as pd

In [2]:
top_delegates_df = pd.read_csv("../Data/Active_Delegates_Data.csv")

In [3]:
top_delegates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   delegate      500 non-null    object 
 1   voting_power  500 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB


In [4]:
top_delegates_df.head()

,delegate,voting_power
0,0x3eee61b92c36e97be6319bf9096a1ac3c04a1466,1.105058e+07
1,0x2ac5393d1f4be4ef89b45ee2f93d7f20a5cf6d5a,7.026108e+06
2,0x06ad892ce23c136bbda3a821570343a2af3e2914,5.557493e+06
3,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,5.221222e+06
4,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,5.041244e+06


In [5]:
votes_df = pd.read_csv("../Data/Agora_Votes_Data.csv")

In [6]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112560 entries, 0 to 112559
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   proposalId  112560 non-null  object 
 1   address     112560 non-null  object 
 2   timestamp   112560 non-null  object 
 3   support     112560 non-null  object 
 4   weight      112560 non-null  float64
dtypes: float64(1), object(4)
memory usage: 4.3+ MB


In [7]:
votes_df.head()

,proposalId,address,timestamp,support,weight
0,1099608625850195404769236791570123626409652639...,0x60ec4fd8069513f738f3a0f41b9e00c294e74bf3,2025-04-24T18:58:45.000Z,FOR,616478.541200
1,1099608625850195404769236791570123626409652639...,0x010dc5440ad49f9ec0dd325b622d9fd225944ee4,2025-04-24T18:22:19.000Z,FOR,121166.613047
2,1099608625850195404769236791570123626409652639...,0x0331969e189d63fbc31d771bb04ab44227d748d8,2025-04-24T19:02:11.000Z,FOR,106811.043504
3,1099608625850195404769236791570123626409652639...,0x0edfcf9bb8b59b07e53b9b1c4d92351e8d922ab8,2025-04-24T19:26:49.000Z,FOR,736.569032
4,1099608625850195404769236791570123626409652639...,0xbe5afc63dc7b7dcd95631d6e37bbe44e9672a9df,2025-04-24T19:13:37.000Z,FOR,676.288173


In [8]:
# unique proposals
votes_df['proposalId'].unique()

array(['109960862585019540476923679157012362640965263962961594700088102558430487321742',
       '38506287861710446593663598830868940900144818754960277981092485594195671514829',
       '105196850607896626370893604768027381433548036180811365072963268567142002370039',
       '84511922734478887667300419900648701566511387783615524992018614345859900443455',
       '8705916809146420472067303211131851783087744913535435360574720946039078686841',
       '95528263587371532982719325402371584327430753545162858644972401153516332664853',
       '71928632649116715308847337447543955907072794738294227130170691217045092512147',
       '31049359136632781771607732021569520613741907517136820917236339424553298132866',
       '3505139576575581948952533286313165208104296221987341923460133599388956364165',
       '91828175807003897805022589352934414611981179534075907474227649658339164272992'],
      dtype=object)

In [9]:
# Ensure consistent address formatting
top_delegates_df['delegate'] = top_delegates_df['delegate'].str.lower()
votes_df['address'] = votes_df['address'].str.lower()

# Filter votes_df to include only top delegates
filtered_votes = votes_df[votes_df['address'].isin(top_delegates_df['delegate'])]

# Count number of unique proposals each delegate voted on
activity_df = (
    filtered_votes.groupby('address')['proposalId']
    .nunique()
    .reset_index()
    .rename(columns={'address': 'delegate', 'proposalId': 'proposal_participation_count'})
)

# Merge with top_delegates_df to include voting_power, even for inactive ones
final_activity = top_delegates_df.merge(activity_df, on='delegate', how='left')
final_activity['proposal_participation_count'] = final_activity['proposal_participation_count'].fillna(0).astype(int)

# Add percentage participatio column
final_activity['participation_percentage'] = (
    final_activity['proposal_participation_count'] / 10 * 100
).round(2)

# Sort by most active
final_activity = final_activity.sort_values(by='proposal_participation_count', ascending=False)


In [10]:
# Display result
final_activity[['delegate', 'voting_power', 'proposal_participation_count', 'participation_percentage']]

,delegate,voting_power,proposal_participation_count,participation_percentage
425,0xd115edee06578716cb235fc8b51bd652b8237dc2,4068.158833,9,90.0
370,0x65d43a62d60f38292a43b9b8ddc0155574db9b62,5161.690514,9,90.0
377,0xfac890eb4c056323441ec68cf4528e3f3462e16c,5047.764521,9,90.0
251,0x87950049c98115095496c28eeb8959d9a4ea9752,9441.823696,9,90.0
234,0x8c57a2288bd9d1a4e02114195678d8eaa215a7a1,10517.215481,9,90.0
...,...,...,...,...
490,0xfafb325429351555f9a4dfba846a3c1707657046,3078.456700,0,0.0
489,0xa1455e19c297b9a0a0e894208d041ad40cc5c23d,3100.864870,0,0.0
488,0xc0b53221a7dbcbce68441c26ea4da73266d6bc20,3105.686319,0,0.0
487,0xa4776cf1eeabed027d27090548dfac37e26da2f6,3109.531642,0,0.0


In [11]:
# Count how many delegates did not vote on any proposals
inactive_delegates = final_activity[final_activity['proposal_participation_count'] == 0]

# Total count
num_inactive = len(inactive_delegates)

print(f"Number of top 500 delegates who did NOT vote on any of the proposals: {num_inactive}")

Number of top 500 delegates who did NOT vote on any of the proposals: 279


In [12]:
inactive_delegates[['delegate', 'voting_power']].head()

,delegate,voting_power
467,0xd836da33a6629548271eb9ef4f62d083376eb4a6,3.316256e+03
3,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,5.221222e+06
5,0xeff8d84e0fd304550da242040ccd45bd44ce71f1,5.004885e+06
9,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,2.593659e+06
480,0x1d19da85322c5f14201be546c326e0e6f521b6e6,3.151565e+03
